In [ ]:
!/opt/bin/nvidia-smi

Wed Oct 27 17:18:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
os.chdir('/content/gdrive/My Drive/Shanghai/')

In [ ]:
import argparse
import torch
import os
import numpy as np
import datasets.crowd as crowd
from models import vgg19

In [ ]:
device = 0
crop_size = 512
model_path = './data/model_sh_B.pth'
data_path = './data/ShanghaiTech/part_B/'
dataset = 'shb'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device('cuda')
dataset = crowd.Crowd_sh(os.path.join(data_path, 'test_data'), crop_size, 8, method='val')
dataloader = torch.utils.data.DataLoader(dataset, 1, shuffle=False,
                                         num_workers=1, pin_memory=True)

number of img: 316


In [ ]:
model = vgg19()
model.to(device)
model.load_state_dict(torch.load(model_path, device))
model.eval()
image_errs = []

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
def interactive_loss(density, gt_density, mask, count_limit = 4):
    density = density * mask
    gt_density = gt_density * mask
    if gt_density.sum() >= count_limit:
        loss = max(0, count_limit - density.sum())
    else:
        loss = max(0, gt_density.sum() - density.sum()) + max(0, density.sum() - gt_density.sum())
    return loss

def sample_pixel(label):
  height, width = label.shape
  y = random.randint(0, height - 1)
  x = random.randint(0, width - 1)
  return y,x

def random_region_sample(label):
  max_label = np.max(label) + 1
  random_label = np.random.randint(0, int(max_label))
  return random_label

def error_region_sample(label, density, gt_density):
  max_label = np.max(label) + 1
  max_error = 0
  final_sample_label = None
  #For all region, calculate the loss
  for sample_label in range(max_label):
    #The Mask
    inter_mask = np.zeros((label.shape[0], label.shape[1]), dtype=np.uint8)
    inter_mask[label == sample_label] = 1
    inter_mask = torch.from_numpy(inter_mask).to(device)
    #Error
    inter_error = interactive_loss(density, gt_density, inter_mask)
    #Max error
    if max_error < inter_error:
      max_error = inter_error
      final_sample_label = sample_label
  return final_sample_label

#Random

##Mean Region Loss

In [ ]:
import copy
import scipy.io as sio
import torch.optim as optim
from region_sum_less_4 import VIS
import torch.nn.functional as F
image_errs = []
mat_path = './data/ShanghaiTech/part_B/test_data/ground-truth/'
INTER_TIMES = 1
INGS = 20
INLR = 5e-8
Inter_result = [[], [], [], [], []]
def adapted_test():
  image_errs = []
  for inputs, count, name in dataloader:
      inputs = inputs.to(device)

      #Get GT_density(Same Size with output)
      density_path = 'GT_' + name[0] + '.mat'
      density_path = os.path.join(mat_path, density_path)
      keypoints = sio.loadmat(density_path)['image_info'][0][0][0][0][0]
      outputs, _ = model(inputs)
      density = outputs.squeeze()
      h, w = density.shape[0], density.shape[1]
      GT_density = np.zeros((h, w))
      for i in range(keypoints.shape[0]):
        x, y = min(int(np.rint(keypoints[i][0] / 8)), w - 1), min(int(np.rint(keypoints[i][1] / 8)), h - 1)
        GT_density[y][x] += 1

      assert GT_density.sum() == count[0].item()
      GT_density = torch.from_numpy(GT_density).to(device)    
      adapted_model = copy.deepcopy(model)
      adapted_model.train()
      '''
      Start Interaction
      '''
      inter_mask_list = []
      for inter_time in range(INTER_TIMES):
        
        assert inputs.size(0) == 1, 'the batch size should equal to 1'
        outputs, _ = adapted_model(inputs)

        #Vis
        density = outputs.squeeze().detach().cpu().numpy()
        visual = VIS(density)
        visual.solve()
        label = visual.Llabel
        
        #Sample Region Randomly
        sample_label = random_region_sample(label)

        #Get Mask
        inter_mask = np.zeros((label.shape[0], label.shape[1]), dtype=np.uint8)
        inter_mask[label == sample_label] = 1
        inter_mask = torch.from_numpy(inter_mask).to(device)
        inter_mask_list.append(inter_mask)
        optimizer_inter = optim.Adam(adapted_model.parameters(), lr=INLR)
        for step in range(0, INGS):
          #Inter Opti
          optimizer_inter.zero_grad()
          output, _ = adapted_model(inputs)
          inter_loss = 0
          #Scheme 2
          for inmask in inter_mask_list:
            inter_loss += interactive_loss(output, GT_density, inmask)
          inter_loss /= len(inter_mask_list)
          #inter_loss = criterion((output * inter_mask).sum(), (gt_density * inter_mask).sum())
          if torch.is_tensor(inter_loss):
              inter_loss.backward()
              optimizer_inter.step()
        
        '''
        Final Test it
        '''
        #with torch.set_grad_enabled(False):
        outputs, _ = adapted_model(inputs)
        img_err = count[0].item() - torch.sum(outputs).item()
      #print(name, img_err, count[0].item(), torch.sum(outputs).item())
      #image_errs.append(img_err)
        Inter_result[inter_time].append(img_err)
  for inter_time in range(INTER_TIMES):
    image_errs = Inter_result[inter_time]
    image_errs = np.array(image_errs)
    mse = np.sqrt(np.mean(np.square(image_errs)))
    mae = np.mean(np.abs(image_errs))
    print('{}: mae {}, mse {}\n'.format(model_path, mae, mse))

INGS = 30
INLR = 1e-8
INTER_TIMES = 5
#for INTER_TIMES in [0, 1, 2, 3, 4, 5]:
#for INGS in [20, 25, 30]:
#  for INLR in [1e-8, 5e-8, 1e-7]:
#    print('############################################################')
#    print('GS: ', INGS, 'LR: ', INLR)
adapted_test()
#59.7 95.7
#Not that much improve since the downsampling 
#Different hyper parameters
#Not that genalize
#7.4 11.8

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3825: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Up

./data/model_sh_B.pth: mae 7.332228150548814, mse 11.828073230453812

./data/model_sh_B.pth: mae 7.364444756809669, mse 11.901028011258349

./data/model_sh_B.pth: mae 7.4014521912683415, mse 11.987576632026256

./data/model_sh_B.pth: mae 7.456820726394653, mse 12.116225423291693

./data/model_sh_B.pth: mae 7.4892718369447735, mse 12.22243840561547



##Sum Region Loss

In [ ]:
import copy
import scipy.io as sio
import torch.optim as optim
from region_sum_less_4 import VIS
import torch.nn.functional as F
image_errs = []
mat_path = './data/ShanghaiTech/part_B/test_data/ground-truth/'
INTER_TIMES = 1
INGS = 20
INLR = 5e-8
Inter_result = [[], [], [], [], []]
def adapted_test():
  image_errs = []
  for inputs, count, name in dataloader:
      inputs = inputs.to(device)

      #Get GT_density(Same Size with output)
      density_path = 'GT_' + name[0] + '.mat'
      density_path = os.path.join(mat_path, density_path)
      keypoints = sio.loadmat(density_path)['image_info'][0][0][0][0][0]
      outputs, _ = model(inputs)
      density = outputs.squeeze()
      h, w = density.shape[0], density.shape[1]
      GT_density = np.zeros((h, w))
      for i in range(keypoints.shape[0]):
        x, y = min(int(np.rint(keypoints[i][0] / 8)), w - 1), min(int(np.rint(keypoints[i][1] / 8)), h - 1)
        GT_density[y][x] += 1

      assert GT_density.sum() == count[0].item()
      GT_density = torch.from_numpy(GT_density).to(device)    
      adapted_model = copy.deepcopy(model)
      adapted_model.train()
      '''
      Start Interaction
      '''
      inter_mask_list = []
      for inter_time in range(INTER_TIMES):
        
        assert inputs.size(0) == 1, 'the batch size should equal to 1'
        outputs, _ = adapted_model(inputs)

        #Vis
        density = outputs.squeeze().detach().cpu().numpy()
        visual = VIS(density)
        visual.solve()
        label = visual.Llabel
        
        #Sample Region Randomly
        sample_label = random_region_sample(label)

        #Get Mask
        inter_mask = np.zeros((label.shape[0], label.shape[1]), dtype=np.uint8)
        inter_mask[label == sample_label] = 1
        inter_mask = torch.from_numpy(inter_mask).to(device)
        inter_mask_list.append(inter_mask)
        optimizer_inter = optim.Adam(adapted_model.parameters(), lr=INLR)
        for step in range(0, INGS):
          #Inter Opti
          optimizer_inter.zero_grad()
          output, _ = adapted_model(inputs)
          inter_loss = 0
          #Scheme 2
          for inmask in inter_mask_list:
            inter_loss += interactive_loss(output, GT_density, inmask)
          #inter_loss /= len(inter_mask_list)
          #inter_loss = criterion((output * inter_mask).sum(), (gt_density * inter_mask).sum())
          if torch.is_tensor(inter_loss):
              inter_loss.backward()
              optimizer_inter.step()
        
        '''
        Final Test it
        '''
        #with torch.set_grad_enabled(False):
        outputs, _ = adapted_model(inputs)
        img_err = count[0].item() - torch.sum(outputs).item()
      #print(name, img_err, count[0].item(), torch.sum(outputs).item())
      #image_errs.append(img_err)
        Inter_result[inter_time].append(img_err)
  for inter_time in range(INTER_TIMES):
    image_errs = Inter_result[inter_time]
    image_errs = np.array(image_errs)
    mse = np.sqrt(np.mean(np.square(image_errs)))
    mae = np.mean(np.abs(image_errs))
    print('{}: mae {}, mse {}\n'.format(model_path, mae, mse))

INGS = 30
INLR = 1e-8
INTER_TIMES = 5
#for INTER_TIMES in [0, 1, 2, 3, 4, 5]:
#for INGS in [20, 25, 30]:
#  for INLR in [1e-8, 5e-8, 1e-7]:
#    print('############################################################')
#    print('GS: ', INGS, 'LR: ', INLR)
adapted_test()
#59.7 95.7
#Not that much improve since the downsampling 
#Different hyper parameters
#Not that genalize

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3825: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


./data/model_sh_B.pth: mae 7.320633459694778, mse 11.778565257057942

./data/model_sh_B.pth: mae 7.328646629671507, mse 11.77471430722256

./data/model_sh_B.pth: mae 7.328259733658802, mse 11.763424049483868

./data/model_sh_B.pth: mae 7.326949131639698, mse 11.772856396240494

./data/model_sh_B.pth: mae 7.348822367342213, mse 11.809955288914995



#Error

##Mean Region Loss

In [ ]:
import copy
import scipy.io as sio
import torch.optim as optim
from region_sum_less_4 import VIS
import torch.nn.functional as F
image_errs = []
mat_path = './data/ShanghaiTech/part_B/test_data/ground-truth/'
INTER_TIMES = 1
INGS = 20
INLR = 5e-8
Inter_result = [[], [], [], [], []]
def adapted_test():
  image_errs = []
  for inputs, count, name in dataloader:
      inputs = inputs.to(device)

      #Get GT_density(Same Size with output)
      density_path = 'GT_' + name[0] + '.mat'
      density_path = os.path.join(mat_path, density_path)
      keypoints = sio.loadmat(density_path)['image_info'][0][0][0][0][0]
      outputs, _ = model(inputs)
      density = outputs.squeeze()
      h, w = density.shape[0], density.shape[1]
      GT_density = np.zeros((h, w))
      for i in range(keypoints.shape[0]):
        x, y = min(int(np.rint(keypoints[i][0] / 8)), w - 1), min(int(np.rint(keypoints[i][1] / 8)), h - 1)
        GT_density[y][x] += 1

      assert GT_density.sum() == count[0].item()
      GT_density = torch.from_numpy(GT_density).to(device)    
      adapted_model = copy.deepcopy(model)
      adapted_model.train()
      '''
      Start Interaction
      '''
      inter_mask_list = []
      for inter_time in range(INTER_TIMES):
        
        assert inputs.size(0) == 1, 'the batch size should equal to 1'
        outputs, _ = adapted_model(inputs)

        #Vis
        density = outputs.squeeze().detach().cpu().numpy()
        visual = VIS(density)
        visual.solve()
        label = visual.Llabel
        
        #Sample Region Randomly
        #sample_label = random_region_sample(label)
        sample_label = error_region_sample(label, outputs.squeeze(), GT_density)
        #Get Mask
        inter_mask = np.zeros((label.shape[0], label.shape[1]), dtype=np.uint8)
        inter_mask[label == sample_label] = 1
        inter_mask = torch.from_numpy(inter_mask).to(device)
        inter_mask_list.append(inter_mask)
        optimizer_inter = optim.Adam(adapted_model.parameters(), lr=INLR)
        for step in range(0, INGS):
          #Inter Opti
          optimizer_inter.zero_grad()
          output, _ = adapted_model(inputs)
          inter_loss = 0
          #Scheme 2
          for inmask in inter_mask_list:
            inter_loss += interactive_loss(output, GT_density, inmask)
          inter_loss /= len(inter_mask_list)
          #inter_loss = criterion((output * inter_mask).sum(), (gt_density * inter_mask).sum())
          if torch.is_tensor(inter_loss):
              inter_loss.backward()
              optimizer_inter.step()
        
        '''
        Final Test it
        '''
        #with torch.set_grad_enabled(False):
        outputs, _ = adapted_model(inputs)
        img_err = count[0].item() - torch.sum(outputs).item()
      #print(name, img_err, count[0].item(), torch.sum(outputs).item())
      #image_errs.append(img_err)
        Inter_result[inter_time].append(img_err)
  for inter_time in range(INTER_TIMES):
    image_errs = Inter_result[inter_time]
    image_errs = np.array(image_errs)
    mse = np.sqrt(np.mean(np.square(image_errs)))
    mae = np.mean(np.abs(image_errs))
    print('{}: mae {}, mse {}\n'.format(model_path, mae, mse))

INGS = 50
INLR = 1e-8
INTER_TIMES = 5
#for INTER_TIMES in [0, 1, 2, 3, 4, 5]:
#for INGS in [20, 25, 30]:
#  for INLR in [1e-8, 5e-8, 1e-7]:
#    print('############################################################')
#    print('GS: ', INGS, 'LR: ', INLR)
adapted_test()
#59.7 95.7
#Not that much improve since the downsampling 
#Different hyper parameters
#Not that genalize

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3825: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Up

./data/model_sh_B.pth: mae 7.179282330259492, mse 11.50696717101904

./data/model_sh_B.pth: mae 7.102348994605149, mse 11.40135704836044

./data/model_sh_B.pth: mae 7.067782773247248, mse 11.28776276030665

./data/model_sh_B.pth: mae 7.05219850962675, mse 11.243700714229947

./data/model_sh_B.pth: mae 7.078190752222568, mse 11.294299804902643



##Sum Region Loss

In [ ]:
import copy
import scipy.io as sio
import torch.optim as optim
from region_sum_less_4 import VIS
import torch.nn.functional as F
image_errs = []
mat_path = './data/ShanghaiTech/part_B/test_data/ground-truth/'
INTER_TIMES = 1
INGS = 20
INLR = 5e-8
Inter_result = [[], [], [], [], []]
def adapted_test():
  image_errs = []
  for inputs, count, name in dataloader:
      inputs = inputs.to(device)

      #Get GT_density(Same Size with output)
      density_path = 'GT_' + name[0] + '.mat'
      density_path = os.path.join(mat_path, density_path)
      keypoints = sio.loadmat(density_path)['image_info'][0][0][0][0][0]
      outputs, _ = model(inputs)
      density = outputs.squeeze()
      h, w = density.shape[0], density.shape[1]
      GT_density = np.zeros((h, w))
      for i in range(keypoints.shape[0]):
        x, y = min(int(np.rint(keypoints[i][0] / 8)), w - 1), min(int(np.rint(keypoints[i][1] / 8)), h - 1)
        GT_density[y][x] += 1

      assert GT_density.sum() == count[0].item()
      GT_density = torch.from_numpy(GT_density).to(device)    
      adapted_model = copy.deepcopy(model)
      adapted_model.train()
      '''
      Start Interaction
      '''
      inter_mask_list = []
      for inter_time in range(INTER_TIMES):
        
        assert inputs.size(0) == 1, 'the batch size should equal to 1'
        outputs, _ = adapted_model(inputs)

        #Vis
        density = outputs.squeeze().detach().cpu().numpy()
        visual = VIS(density)
        visual.solve()
        label = visual.Llabel
        
        #Sample Region Randomly
        #sample_label = random_region_sample(label)
        sample_label = error_region_sample(label, outputs.squeeze(), GT_density)
        #Get Mask
        inter_mask = np.zeros((label.shape[0], label.shape[1]), dtype=np.uint8)
        inter_mask[label == sample_label] = 1
        inter_mask = torch.from_numpy(inter_mask).to(device)
        inter_mask_list.append(inter_mask)
        optimizer_inter = optim.Adam(adapted_model.parameters(), lr=INLR)
        for step in range(0, INGS):
          #Inter Opti
          optimizer_inter.zero_grad()
          output, _ = adapted_model(inputs)
          inter_loss = 0
          #Scheme 2
          for inmask in inter_mask_list:
            inter_loss += interactive_loss(output, GT_density, inmask)
          #inter_loss /= len(inter_mask_list)
          #inter_loss = criterion((output * inter_mask).sum(), (gt_density * inter_mask).sum())
          if torch.is_tensor(inter_loss):
              inter_loss.backward()
              optimizer_inter.step()
        
        '''
        Final Test it
        '''
        #with torch.set_grad_enabled(False):
        outputs, _ = adapted_model(inputs)
        img_err = count[0].item() - torch.sum(outputs).item()
      #print(name, img_err, count[0].item(), torch.sum(outputs).item())
      #image_errs.append(img_err)
        Inter_result[inter_time].append(img_err)
  for inter_time in range(INTER_TIMES):
    image_errs = Inter_result[inter_time]
    image_errs = np.array(image_errs)
    mse = np.sqrt(np.mean(np.square(image_errs)))
    mae = np.mean(np.abs(image_errs))
    print('{}: mae {}, mse {}\n'.format(model_path, mae, mse))

INGS = 50
INLR = 1e-8
INTER_TIMES = 5
#for INTER_TIMES in [0, 1, 2, 3, 4, 5]:
#for INGS in [20, 25, 30]:
#  for INLR in [1e-8, 5e-8, 1e-7]:
#    print('############################################################')
#    print('GS: ', INGS, 'LR: ', INLR)
adapted_test()
#59.7 95.7
#Not that much improve since the downsampling 
#Different hyper parameters
#Not that genalize

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3825: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


./data/model_sh_B.pth: mae 7.1792921899240225, mse 11.50697292621664

./data/model_sh_B.pth: mae 7.102370729929285, mse 11.40137532249873

./data/model_sh_B.pth: mae 7.0678221575821505, mse 11.287805949227725

./data/model_sh_B.pth: mae 7.0522789019572585, mse 11.243767637490741

./data/model_sh_B.pth: mae 7.078279706496227, mse 11.294367267311607

